In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,count,explode,col,monotonically_increasing_id

spark = SparkSession.builder \
    .appName("fixture_demo") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/02/08 14:49:02 WARN Utils: Your hostname, DESKTOP-PR49LJH resolves to a loopback address: 127.0.1.1; using 192.168.254.36 instead (on interface eth0)
24/02/08 14:49:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/08 14:49:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_fixtures = spark.read.json("file:///home/jss/app/spark/data/soccer_data/fixture/2024-01-28.json", multiLine=True)
# "response" 및 "element" 열 확장
df_fixture = df_fixtures.select(
    explode("response.fixture").alias("expanded_fixture"),
    monotonically_increasing_id().alias("index_fixture")
    
)
df_goals = df_fixtures.select(explode("response.goals").alias("expanded_goals"),
                             monotonically_increasing_id().alias("index_goals"))


df_fixture_goals = df_fixture.join(df_goals,df_fixture['index_fixture']==df_goals['index_goals'],"inner")
df_league = df_fixtures.select(
    explode("response.league").alias("expanded_league"),
     monotonically_increasing_id().alias("index_league")
)

df_league_fixture =df_fixture_goals.join(df_league,df_fixture_goals['index_fixture']==df_league['index_league'],"inner")

df_teams=df_fixtures.select(
    explode("response.teams").alias("expanded_teams"),
     monotonically_increasing_id().alias("index_teams")
    
)

df_team_fixture=df_league_fixture.join(df_teams,df_league_fixture['index_fixture']==df_teams['index_teams'],'inner')

# "expanded_element"에서 필요한 정보 추출
df_result = df_team_fixture.select(
    "expanded_fixture.date",
    df_fixture["expanded_fixture.id"].alias("fixture_id"),
    "expanded_fixture.periods.first",
    "expanded_fixture.periods.second",
    "expanded_fixture.timestamp",
    "expanded_fixture.referee",
    df_fixture["expanded_fixture.venue.id"].alias("venue_id"),
    "expanded_goals.home",
    "expanded_goals.away",
    df_league["expanded_league.id"].alias("league_id"),
    "expanded_league.round",
    "expanded_league.season",
    df_teams["expanded_teams.home.id"].alias("home_team_id"),
    df_teams["expanded_teams.away.id"].alias("away_team_id") 
)

        #.selectExpr("fixture.id",
        #            "fixture.refree",
        #            "fixture.date",
        #            "fixture.timestamp")

In [3]:
df_result.show()

+--------------------+----------+----------+----------+----------+--------------------+--------+----+----+---------+-------------------+------+------------+------------+
|                date|fixture_id|     first|    second| timestamp|             referee|venue_id|home|away|league_id|              round|season|home_team_id|away_team_id|
+--------------------+----------+----------+----------+----------+--------------------+--------+----+----+---------+-------------------+------+------------+------------+
|2024-01-28T16:30:...|   1049043|1706459400|1706463000|1706459400|Benjamin Brand, G...|   19381|   3|   1|       78|Regular Season - 19|  2023|         165|         176|
|2024-01-28T14:30:...|   1049044|1706452200|1706455800|1706452200|Harm Osmers, Germany|     748|   1|   0|       78|Regular Season - 19|  2023|         182|         181|
+--------------------+----------+----------+----------+----------+--------------------+--------+----+----+---------+-------------------+------+-------

In [4]:
df_fixtures.createOrReplaceTempView("df_fixtures")

In [82]:
df=spark.sql("""WITH
    expanded_fixture AS (
        SELECT
            explode(response.fixture) as expanded_fixture,
            monotonically_increasing_id() as index_fixture 
        FROM
            df_fixtures
    )
             ,
    expanded_goals AS (
        SELECT
            explode(response.goals) expanded_goals,
            monotonically_increasing_id() AS index_goals
        FROM
            df_fixtures
    ),
    expanded_league AS (
        SELECT
            explode(response.league) as expanded_league,
            monotonically_increasing_id() AS index_league
        FROM
            df_fixtures
    ),
    expanded_teams AS (
        SELECT
            explode(response.teams) as expanded_teams,
            monotonically_increasing_id() AS index_teams
        FROM
            df_fixtures),
    df_fixture_goals AS (
        SELECT
            expanded_fixture.*,
            expanded_goals.*
        FROM
            expanded_fixture
            JOIN expanded_goals ON expanded_fixture.index_fixture = expanded_goals.index_goals
    ),
    df_league_fixture AS (
        SELECT
            df_fixture_goals.*,
            expanded_league.*
        FROM
            df_fixture_goals
            JOIN expanded_league ON df_fixture_goals.index_fixture = expanded_league.index_league
    ),
    df_team_fixture AS (
        SELECT
            df_league_fixture.*,
            expanded_teams.*
        FROM
            df_league_fixture
            JOIN expanded_teams ON df_league_fixture.index_fixture = expanded_teams.index_teams
    )
    Select
    df_team_fixture.expanded_fixture.date as date,
    df_team_fixture.expanded_fixture.id AS fixture_id,
    df_team_fixture.expanded_fixture.periods.first AS periods_first,
    df_team_fixture.expanded_fixture.periods.second AS periods_second,
    df_team_fixture.expanded_fixture.timestamp AS timestamp,
    df_team_fixture.expanded_fixture.referee AS referee,
    df_team_fixture.expanded_fixture.venue.id AS venue_id,
    df_team_fixture.expanded_goals.home AS home,
    df_team_fixture.expanded_goals.away AS away,
    df_team_fixture.expanded_league.id AS league_id,
    df_team_fixture.expanded_league.round AS round,
    df_team_fixture.expanded_league.season AS season,
    df_team_fixture.expanded_teams.home.id AS home_team_id,
    df_team_fixture.expanded_teams.away.id AS away_team_id       
FROM
    df_team_fixture;
   
""")


In [83]:
df.show()

+--------------------+----------+-------------+--------------+----------+--------------------+--------+----+----+---------+-------------------+------+------------+------------+
|                date|fixture_id|periods_first|periods_second| timestamp|             referee|venue_id|home|away|league_id|              round|season|home_team_id|away_team_id|
+--------------------+----------+-------------+--------------+----------+--------------------+--------+----+----+---------+-------------------+------+------------+------------+
|2024-01-28T16:30:...|   1049043|   1706459400|    1706463000|1706459400|Benjamin Brand, G...|   19381|   3|   1|       78|Regular Season - 19|  2023|         165|         176|
|2024-01-28T14:30:...|   1049044|   1706452200|    1706455800|1706452200|Harm Osmers, Germany|     748|   1|   0|       78|Regular Season - 19|  2023|         182|         181|
+--------------------+----------+-------------+--------------+----------+--------------------+--------+----+----+--

In [ ]:
df_fixtures.printSchema()

root
 |-- errors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- get: string (nullable = true)
 |-- paging: struct (nullable = true)
 |    |-- current: long (nullable = true)
 |    |-- total: long (nullable = true)
 |-- parameters: struct (nullable = true)
 |    |-- from: string (nullable = true)
 |    |-- league: string (nullable = true)
 |    |-- season: string (nullable = true)
 |    |-- to: string (nullable = true)
 |-- response: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fixture: struct (nullable = true)
 |    |    |    |-- date: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- periods: struct (nullable = true)
 |    |    |    |    |-- first: long (nullable = true)
 |    |    |    |    |-- second: long (nullable = true)
 |    |    |    |-- referee: string (nullable = true)
 |    |    |    |-- status: struct (nullable = true)
 |    |    |    |    |-- elapsed: long (n

In [3]:
df_team_statistics = spark.read.json("file:///home/jss/app/spark/data/soccer_data/team_statistics/team_statistics_sample.json", multiLine=True)

df_team_league = df_team_statistics.select(
    col("response.league.id").alias("league_id"),
    col("response.league.season").alias("season"),
    monotonically_increasing_id().alias("index_league")
)
df_team= df_team_statistics.select( col("response.team.id").alias("team_id"),
    monotonically_increasing_id().alias("index_team"))


df_league_team = df_team_league.join(df_team,df_team_league['index_league']==df_team['index_team'],"inner")
df_form= df_team_statistics.select( col("response.form").alias("team_form"),
                                   monotonically_increasing_id().alias("index_form"))
df_league_form = df_league_team.join(df_form,df_league_team["index_league"]==df_form['index_form'],"inner")
df_fixtures = df_team_statistics.select(
    col("response.fixtures.played.home").alias("home_game"),
    col("response.fixtures.played.away").alias("away_game"),
    col("response.fixtures.played.total").alias("total_game"),
    col("response.fixtures.wins.home").alias("home_wins"),
    col("response.fixtures.wins.away").alias("away_wins"),
    col("response.fixtures.wins.total").alias("total_wins"),
    col("response.fixtures.draws.home").alias("home_draws"),
    col("response.fixtures.draws.away").alias("away_draws"),
    col("response.fixtures.draws.total").alias("total_draws"),
    col("response.fixtures.loses.home").alias("home_loses"),
    col("response.fixtures.loses.away").alias("away_loses"),
    col("response.fixtures.loses.total").alias("total_loses"),
    monotonically_increasing_id().alias("index_fixtures")
 )

df_league_fixture =df_league_form.join(df_fixtures,df_league_form['index_league']==df_fixtures['index_fixtures'],"inner")

df_goals=df_team_statistics.select(
     col("response.goals.for.total.home").alias("home_goals"),
     col("response.goals.for.total.away").alias("away_goals"),
     col("response.goals.for.total.total").alias("total_goals"),
     col("response.goals.for.average.home").alias("home_avg_goals"),
     col("response.goals.for.average.away").alias("away_avg_goals"),
     col("response.goals.for.average.total").alias("total_avg_goals"),
     col("response.goals.for.minute.0-15.total").alias("0-15_total_goals"),
     col("response.goals.for.minute.0-15.percentage").alias("0-15_goals_percentage"),
    col("response.goals.for.minute.16-30.total").alias("16-30_total_goals"),
    col("response.goals.for.minute.16-30.percentage").alias("16-30_goals_percentage"),
    col("response.goals.for.minute.31-45.total").alias("31-45_total_goals"),
    col("response.goals.for.minute.31-45.percentage").alias("31-45_goals_percentage"),
    col("response.goals.for.minute.46-60.total").alias("46-60_total_goals"),
    col("response.goals.for.minute.46-60.percentage").alias("46-60_goals_percentage"),
    col("response.goals.for.minute.61-75.total").alias("61-75_total_goals"),
    col("response.goals.for.minute.61-75.percentage").alias("61-75_goals_percentage"),
    col("response.goals.for.minute.76-90.total").alias("76-90_total_goals"),
    col("response.goals.for.minute.76-90.percentage").alias("76-90_goals_percentage"),
    col("response.goals.for.minute.91-105.total").alias("91-105_total_goals"),
    col("response.goals.for.minute.91-105.percentage").alias("91-105_goals_percentage"),
    col("response.goals.for.minute.106-120.total").alias("106-120_total_goals"),
    col("response.goals.for.minute.106-120.percentage").alias("106-120_goals_percentage"),
    col("response.goals.against.minute.0-15.percentage").alias("0-15_against_percentage"),
    col("response.goals.against.minute.16-30.total").alias("16-30_total_against_goals"),
    col("response.goals.against.minute.16-30.percentage").alias("16-30_against_percentage"),
    col("response.goals.against.minute.31-45.total").alias("31-45_total_againsT_goals"),
    col("response.goals.against.minute.31-45.percentage").alias("31-45_against_percentage"),
    col("response.goals.against.minute.46-60.total").alias("46-60_total_against_goals"),
    col("response.goals.against.minute.46-60.percentage").alias("46-60_against_percentage"),
    col("response.goals.against.minute.61-75.total").alias("61-75_total_against_goals"),
    col("response.goals.against.minute.61-75.percentage").alias("61-75_against_percentage"),
    col("response.goals.against.minute.76-90.total").alias("76-90_total_against_goals"),
    col("response.goals.against.minute.76-90.percentage").alias("76-90_against_percentage"),
    col("response.goals.against.minute.91-105.total").alias("91-105_total_against_goals"),
    col("response.goals.against.minute.91-105.percentage").alias("91-105_against_percentage"),
    col("response.goals.against.minute.106-120.total").alias("106-120_total_against_goals"),
    col("response.goals.against.minute.106-120.percentage").alias("106-120_against_percentage"),
    monotonically_increasing_id().alias("index_goals")
)

df_fixture_goals=df_league_fixture.join(df_goals,df_league_fixture['index_fixtures']==df_goals['index_goals'],'inner')

df_biggest=df_team_statistics.select(
     col("response.biggest.streak.wins").alias("biggest_streak_wins"),
     col("response.biggest.streak.draws").alias("biggest_streak_draws"),
     col("response.biggest.streak.loses").alias("biggest_streak_loses"),
     col("response.biggest.wins.home").alias("biggest_home_wins"),
     col("response.biggest.wins.away").alias("biggest_away_wins"),
     col("response.biggest.loses.home").alias("biggest_home_loses"),
     col("response.biggest.loses.away").alias("biggest_away_loses"),
     col("response.biggest.goals.for.home").alias("biggest_home_goals"),
     col("response.biggest.goals.for.away").alias("biggest_away_goals"),
     col("response.biggest.goals.against.home").alias("biggest_home_against"),
     col("response.biggest.goals.against.away").alias("biggest_away_against"),
    monotonically_increasing_id().alias("index_biggest")
    
 )
df_fixture_biggest = df_fixture_goals.join(df_biggest,df_fixture_goals['index_fixtures']==df_biggest['index_biggest'],'inner')

df_clean_sheet=df_team_statistics.select(
      col("response.clean_sheet.home").alias("home_clean_sheet"),
      col("response.clean_sheet.away").alias("away_clean_sheet"),
      col("response.clean_sheet.total").alias("total_clean_sheet"),
      monotonically_increasing_id().alias("index_clean_sheet")
    
)
df_fixture_clean = df_fixture_biggest.join(df_clean_sheet,df_fixture_biggest['index_fixtures']==df_clean_sheet['index_clean_sheet'],'inner')
df_failed_to_score=df_team_statistics.select(
      col("response.failed_to_score.home").alias("home_failed_to_score"),
      col("response.failed_to_score.away").alias("away_failed_to_score"),
      col("response.failed_to_score.total").alias("total_failed_to_score"),
      monotonically_increasing_id().alias("index_failed_to_score")
    
 )
df_fixture_fail_score = df_fixture_clean.join(df_failed_to_score,df_fixture_clean['index_fixtures']==df_failed_to_score['index_failed_to_score'],'inner')

df_penalty=df_team_statistics.select(
      col("response.penalty.scored.total").alias("total_penalty_scored"),
    col("response.penalty.scored.percentage").alias("total_penalty_scored_percentage"),
    col("response.penalty.missed.total").alias("total_penalty_missed"),
    col("response.penalty.missed.percentage").alias("total_penalty_missed_percentage"),
    col("response.penalty.total").alias("total_penalty"),
      monotonically_increasing_id().alias("index_penalty")
 )
df_fixture_penalty = df_fixture_fail_score.join(df_penalty,df_fixture_fail_score['index_fixtures']==df_penalty['index_penalty'],'inner')

# df_lineups=df_team_statistics.select(
#     explode("response.lineups").alias("expanded_lineups"),
#     monotonically_increasing_id().alias("index_lineups")
#  )
# df_lineups.show()
# df_fixture_line_up = df_fixture_penalty.join(df_lineups,df_fixture_penalty['index_fixtures']==df_lineups['index_lineups'],'inner')

df_cards=df_team_statistics.select(
    col("response.cards.yellow.0-15.total").alias("0-15_yellow"),
    col("response.cards.yellow.0-15.percentage").alias("0-15_yellow_percentage"),
    col("response.cards.yellow.16-30.total").alias("16-30_yellow"),
    col("response.cards.yellow.16-30.percentage").alias("16-30_yellow_percentage"),
    col("response.cards.yellow.31-45.total").alias("31-45_yellow"),
    col("response.cards.yellow.31-45.percentage").alias("31-45_yellow_percentage"),
    col("response.cards.yellow.46-60.total").alias("46-60_yellow"),
    col("response.cards.yellow.46-60.percentage").alias("46-60_yellow_percentage"),
    col("response.cards.yellow.61-75.total").alias("61-75_yellow"),
    col("response.cards.yellow.61-75.percentage").alias("61-75_yellow_percentage"),
    col("response.cards.yellow.76-90.total").alias("76-90_yellow"),
    col("response.cards.yellow.76-90.percentage").alias("76-90_yellow_percentage"),
    col("response.cards.yellow.91-105.total").alias("91-105_yellow"),
    col("response.cards.yellow.91-105.percentage").alias("91-105_yellow_percentage"),
    col("response.cards.yellow.106-120.total").alias("106-120_yellows"),
    col("response.cards.yellow.106-120.percentage").alias("106-120_yellow_percentage"),
    col("response.cards.red.0-15.total").alias("0-15_red"),
    col("response.cards.red.0-15.percentage").alias("0-15_red_percentage"),
    col("response.cards.red.16-30.total").alias("16-30_red"),
    col("response.cards.red.16-30.percentage").alias("16-30_red_percentage"),
    col("response.cards.red.31-45.total").alias("31-45_red"),
    col("response.cards.red.31-45.percentage").alias("31-45_red_percentage"),
    col("response.cards.red.46-60.total").alias("46-60_red"),
    col("response.cards.red.46-60.percentage").alias("46-60_red_percentage"),
    col("response.cards.red.61-75.total").alias("61-75_red"),
    col("response.cards.red.61-75.percentage").alias("61-75_red_percentage"),
    col("response.cards.red.76-90.total").alias("76-90_red"),
    col("response.cards.red.76-90.percentage").alias("76-90_red_percentage"),
    col("response.cards.red.91-105.total").alias("91-105_red"),
    col("response.cards.red.91-105.percentage").alias("91-105_red_percentage"),
    col("response.cards.red.106-120.total").alias("106-120_red"),
    col("response.cards.red.106-120.percentage").alias("106-120_red_percentage"),
    monotonically_increasing_id().alias("index_cards")
    
 )
df_fixture_card = df_fixture_penalty.join(df_cards,df_fixture_penalty['index_fixtures']==df_cards['index_cards'],'inner')

# # "expanded_element"에서 필요한 정보 추출
df_result = df_fixture_card.select(
 "index_league", "league_id","season","team_id","team_form","home_game","away_game","total_game","home_wins","away_wins","total_wins","home_draws","away_draws","total_draws","home_loses","away_loses","total_loses","home_goals","away_goals","total_goals","home_avg_goals","away_avg_goals","total_avg_goals","0-15_total_goals","0-15_goals_percentage","16-30_total_goals","16-30_goals_percentage","31-45_total_goals","31-45_goals_percentage","46-60_total_goals","46-60_goals_percentage","61-75_total_goals","61-75_goals_percentage","76-90_total_goals","76-90_goals_percentage","91-105_total_goals","91-105_goals_percentage","106-120_total_goals","106-120_goals_percentage","0-15_against_percentage","16-30_total_against_goals","16-30_against_percentage","31-45_total_against_goals","31-45_against_percentage","46-60_total_against_goals","46-60_against_percentage","61-75_total_against_goals","61-75_against_percentage","76-90_total_against_goals","76-90_against_percentage","91-105_total_against_goals","91-105_against_percentage","106-120_total_against_goals","106-120_against_percentage","biggest_streak_wins","biggest_streak_draws","biggest_streak_loses","biggest_home_wins","biggest_away_wins","biggest_home_loses","biggest_away_loses","biggest_home_goals","biggest_away_goals","biggest_home_against","biggest_away_against","home_clean_sheet","away_clean_sheet","total_clean_sheet","home_failed_to_score","away_failed_to_score","total_failed_to_score","total_penalty_scored","total_penalty_scored_percentage","total_penalty_missed","total_penalty_missed_percentage","total_penalty","0-15_yellow","0-15_yellow_percentage","16-30_yellow","16-30_yellow_percentage","31-45_yellow","31-45_yellow_percentage","46-60_yellow","46-60_yellow_percentage","61-75_yellow","61-75_yellow_percentage","76-90_yellow","76-90_yellow_percentage","91-105_yellow","91-105_yellow_percentage","106-120_yellows","106-120_yellow_percentage","0-15_red","0-15_red_percentage","16-30_red","16-30_red_percentage","31-45_red","31-45_red_percentage","46-60_red","46-60_red_percentage","61-75_red","61-75_red_percentage","76-90_red","76-90_red_percentage","91-105_red","91-105_red_percentage","106-120_red","106-120_red_percentage"

)


In [ ]:
df_stat= spark.sql("""""")

In [4]:
df_team_statistics.printSchema()

root
 |-- errors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- get: string (nullable = true)
 |-- paging: struct (nullable = true)
 |    |-- current: long (nullable = true)
 |    |-- total: long (nullable = true)
 |-- parameters: struct (nullable = true)
 |    |-- league: string (nullable = true)
 |    |-- season: string (nullable = true)
 |    |-- team: string (nullable = true)
 |-- response: struct (nullable = true)
 |    |-- biggest: struct (nullable = true)
 |    |    |-- goals: struct (nullable = true)
 |    |    |    |-- against: struct (nullable = true)
 |    |    |    |    |-- away: long (nullable = true)
 |    |    |    |    |-- home: long (nullable = true)
 |    |    |    |-- for: struct (nullable = true)
 |    |    |    |    |-- away: long (nullable = true)
 |    |    |    |    |-- home: long (nullable = true)
 |    |    |-- loses: struct (nullable = true)
 |    |    |    |-- away: string (nullable = true)
 |    |    |    |-- home: string (nul

In [5]:
df_result.show()

24/02/07 18:17:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+---------+------+-------+--------------------+---------+---------+----------+---------+---------+----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+--------------+--------------+---------------+----------------+---------------------+-----------------+----------------------+-----------------+----------------------+-----------------+----------------------+-----------------+----------------------+-----------------+----------------------+------------------+-----------------------+-------------------+------------------------+-----------------------+-------------------------+------------------------+-------------------------+------------------------+-------------------------+------------------------+-------------------------+------------------------+-------------------------+------------------------+--------------------------+-------------------------+---------------------------+--------------------------+-----------

In [87]:
player_statistics = spark.read.json("file:///home/jss/app/spark/data/soccer_data/player_statistics/player_statistics_sample.json",multiLine=True)

df_player = player_statistics.select(
    explode("response.player").alias("expanded_player"),
    monotonically_increasing_id().alias("index_player")
    
)
player_statistic= player_statistics.select(explode("response.statistics").alias("expanded_statistics"),
                              monotonically_increasing_id().alias("index_statistics"))


df_player_stat= df_player.join(player_statistic,df_player['index_player']==player_statistic['index_statistics'],"inner")

# # "expanded_element"에서 필요한 정보 추출
df_result = df_player_stat.select(
    col("expanded_player.id").alias("player_id"),
    col("expanded_statistics.team.id").getItem(0).alias("team_id"),
    col("expanded_statistics.league.id").getItem(0).alias("league_id"),
    col("expanded_statistics.league.season").getItem(0).alias("league_season"),
    col("expanded_statistics.games.appearences").getItem(0).alias("game_appearances"),
    col("expanded_statistics.games.lineups").getItem(0).alias("game_lineups"),
    col("expanded_statistics.games.minutes").getItem(0).alias("game_minutes"),
    col("expanded_statistics.games.number").getItem(0).alias("game_number"),
    col("expanded_statistics.games.position").getItem(0).alias("game_position"),
    col("expanded_statistics.games.rating").getItem(0).alias("game_rating"),
    col("expanded_statistics.games.captain").getItem(0).alias("game_captain"),
    col("expanded_statistics.substitutes.in").getItem(0).alias("sub_in"),
    col("expanded_statistics.substitutes.out").getItem(0).alias("sub_out"),
    col("expanded_statistics.substitutes.bench").getItem(0).alias("sub_bench"),
    col("expanded_statistics.shots.total").getItem(0).alias("total_shots"),
    col("expanded_statistics.shots.on").getItem(0).alias("shots_on"),
    col("expanded_statistics.goals.total").getItem(0).alias("total_goals"),
    col("expanded_statistics.goals.conceded").getItem(0).alias("goals_conceded"),
    col("expanded_statistics.goals.assists").getItem(0).alias("goals_assists"),
    col("expanded_statistics.goals.saves").getItem(0).alias("goals_saves"),
    col("expanded_statistics.passes.total").getItem(0).alias("passes_total"),
    col("expanded_statistics.passes.key").getItem(0).alias("passes_key"),
    col("expanded_statistics.passes.accuracy").getItem(0).alias("passes_accuracy"),
    col("expanded_statistics.tackles.total").getItem(0).alias("tackles_total"),
    col("expanded_statistics.tackles.blocks").getItem(0).alias("tackles_blocks"),
    col("expanded_statistics.tackles.interceptions").getItem(0).alias("tackles_interceptions"),
    col("expanded_statistics.duels.total").getItem(0).alias("duels_total"),
    col("expanded_statistics.duels.won").getItem(0).alias("duels_won"),
    col("expanded_statistics.dribbles.attempts").getItem(0).alias("dribbles_attempts"),
    col("expanded_statistics.dribbles.success").getItem(0).alias("dribbles_success"),
    col("expanded_statistics.dribbles.past").getItem(0).alias("dribbles_past"),
    col("expanded_statistics.fouls.drawn").getItem(0).alias("fouls_drawn"),
    col("expanded_statistics.fouls.committed").getItem(0).alias("fouls_committed"),
    col("expanded_statistics.cards.yellow").getItem(0).alias("cards_yellow"),
    col("expanded_statistics.cards.yellowred").getItem(0).alias("cards_yellowred"),
    col("expanded_statistics.cards.red").getItem(0).alias("cards_red"),
    col("expanded_statistics.penalty.won").getItem(0).alias("penalty_won"),
    col("expanded_statistics.penalty.commited").getItem(0).alias("penalty_commited"),
    col("expanded_statistics.penalty.scored").getItem(0).alias("penalty_scored"),
    col("expanded_statistics.penalty.missed").getItem(0).alias("penalty_missed"),
    col("expanded_statistics.penalty.saved").getItem(0).alias("penalty_saved")

 )

In [49]:
df_result.show()

+---------+-------+---------+-------------+----------------+------------+------------+-----------+-------------+-----------+------------+------+-------+---------+-----------+--------+-----------+--------------+-------------+-----------+------------+----------+---------------+-------------+--------------+---------------------+-----------+---------+-----------------+----------------+-------------+-----------+---------------+------------+---------------+---------+-----------+----------------+--------------+--------------+-------------+
|player_id|team_id|league_id|league_season|game_appearances|game_lineups|game_minutes|game_number|game_position|game_rating|game_captain|sub_in|sub_out|sub_bench|total_shots|shots_on|total_goals|goals_conceded|goals_assists|goals_saves|passes_total|passes_key|passes_accuracy|tackles_total|tackles_blocks|tackles_interceptions|duels_total|duels_won|dribbles_attempts|dribbles_success|dribbles_past|fouls_drawn|fouls_committed|cards_yellow|cards_yellowred|cards_

In [88]:
player_statistics.createOrReplaceTempView("player_statistics")

In [104]:
play_stat = spark.sql("""WITH
    expanded_players AS (
        SELECT
            explode(response.player) as expanded_player,
            monotonically_increasing_id() as index_player 
        FROM
            player_statistics
    ),
    expanded_statistics AS (
        SELECT
            explode(response.statistics) as expanded_statistics,
            monotonically_increasing_id() AS index_statistics
        FROM
            player_statistics
    ),
        df_player_stat AS (
        SELECT
            expanded_players.*,
            expanded_statistics.*
        FROM
            expanded_players
            JOIN expanded_statistics ON expanded_players.index_player = expanded_statistics.index_statistics
    )
        SELECT
            expanded_player.id as player_id,
            expanded_statistics.team.id[0] as team_id ,
            expanded_statistics.league.id[0] AS league_id,
            expanded_statistics.league.season[0] AS league_season,
            expanded_statistics.games.appearences[0] AS game_appearances,
            expanded_statistics.games.lineups[0] AS game_lineups,
            expanded_statistics.games.minutes[0] AS game_minutes,
            expanded_statistics.games.number[0] AS game_number,
            expanded_statistics.games.position[0] AS game_position,
            expanded_statistics.games.rating[0] AS game_rating,
            expanded_statistics.games.captain[0] AS game_captain,
            expanded_statistics.substitutes.in[0] AS sub_in,
            expanded_statistics.substitutes.out[0] AS sub_out,
            expanded_statistics.substitutes.bench[0] AS sub_bench,
            expanded_statistics.shots.total[0] AS total_shots,
            expanded_statistics.shots.on[0] AS shots_on,
            expanded_statistics.goals.total[0] AS total_goals,
            expanded_statistics.goals.conceded[0] AS goals_conceded,
            expanded_statistics.goals.assists[0] AS goals_assists,
            expanded_statistics.goals.saves[0] AS goals_saves,
            expanded_statistics.passes.total[0] AS passes_total,
            expanded_statistics.passes.key[0] AS passes_key,
            expanded_statistics.passes.accuracy[0] AS passes_accuracy,
            expanded_statistics.tackles.total[0] AS tackles_total,
            expanded_statistics.tackles.blocks[0] AS tackles_blocks,
            expanded_statistics.tackles.interceptions[0] AS tackles_interceptions,
            expanded_statistics.duels.total[0] AS duels_total,
            expanded_statistics.duels.won[0] AS duels_won,
            expanded_statistics.dribbles.attempts[0] AS dribbles_attempts,
            expanded_statistics.dribbles.success[0] AS dribbles_success,
            expanded_statistics.dribbles.past[0] AS dribbles_past,
            expanded_statistics.fouls.drawn[0] AS fouls_drawn,
            expanded_statistics.fouls.committed[0] AS fouls_committed,
            expanded_statistics.cards.yellow[0] AS cards_yellow,
            expanded_statistics.cards.yellowred[0] AS cards_yellowred,
            expanded_statistics.cards.red[0] AS cards_red,
            expanded_statistics.penalty.won[0] AS penalty_won,
            expanded_statistics.penalty.commited[0] AS penalty_commited,
            expanded_statistics.penalty.scored[0] AS penalty_scored,
            expanded_statistics.penalty.missed[0] AS penalty_missed,
            expanded_statistics.penalty.saved[0] AS penalty_saved from df_player_stat""")

In [105]:
play_stat.show()

24/02/08 17:34:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+-------+---------+-------------+----------------+------------+------------+-----------+-------------+-----------+------------+------+-------+---------+-----------+--------+-----------+--------------+-------------+-----------+------------+----------+---------------+-------------+--------------+---------------------+-----------+---------+-----------------+----------------+-------------+-----------+---------------+------------+---------------+---------+-----------+----------------+--------------+--------------+-------------+
|player_id|team_id|league_id|league_season|game_appearances|game_lineups|game_minutes|game_number|game_position|game_rating|game_captain|sub_in|sub_out|sub_bench|total_shots|shots_on|total_goals|goals_conceded|goals_assists|goals_saves|passes_total|passes_key|passes_accuracy|tackles_total|tackles_blocks|tackles_interceptions|duels_total|duels_won|dribbles_attempts|dribbles_success|dribbles_past|fouls_drawn|fouls_committed|cards_yellow|cards_yellowred|cards_

In [106]:
injuries = spark.read.json("file:///home/jss/app/spark/data/soccer_data/injuries/2023-08-11.json",multiLine=True)
df_injuries=injuries.select(col("response.player.id").getItem(0).alias("player_id"),
                            col("response.player.type").getItem(0).alias("injury_type"),
                            col("response.player.reason").getItem(0).alias("injury_resone"),
                            col("response.team.id").getItem(0).alias("team_id"),
                            col("response.fixture.id").getItem(0).alias("fixture_id"),
                            col("response.league.id").getItem(0).alias("league_id")
)


In [107]:
df_injuries.show()

+---------+---------------+------------+-------+----------+---------+
|player_id|    injury_type| injury_type|team_id|fixture_id|league_id|
+---------+---------------+------------+-------+----------+---------+
|    18957|Missing Fixture|Thigh Injury|     44|   1035037|       39|
+---------+---------------+------------+-------+----------+---------+



In [121]:
injuries.createOrReplaceTempView("injury")

In [137]:
injury = spark.sql("""select response.player.id[0] as player_id,
                   response.player.type[0] as injury_type,
                    response.player.reason[0] as injury_reason,
                   response.team.id[0] as team_id,
                   response.fixture.id[0] as fixture_id,
                   response.league.id[0] as league_id from injury
""")

In [138]:
injury.show()

+---------+---------------+-------------+-------+----------+---------+
|player_id|    injury_type|injury_reason|team_id|fixture_id|league_id|
+---------+---------------+-------------+-------+----------+---------+
|    18957|Missing Fixture| Thigh Injury|     44|   1035037|       39|
+---------+---------------+-------------+-------+----------+---------+



In [139]:
player_sideline = spark.read.json("file:///home/jss/app/spark/data/soccer_data/sidelined/sidelined_sample.json",multiLine=True)
df_player_sidelined=player_sideline.select(col("parameters.player").alias("player_id"),
                            explode("response").alias("expanded_sideline"),
    monotonically_increasing_id().alias("index_player") )

df_result = df_player_sidelined.select(
    "player_id",
    col("expanded_sideline.type").alias("sideline_type"),
    col("expanded_sideline.start").alias("sideline_start"),
    col("expanded_sideline.end").alias("sideline_end")
    )

In [140]:
df_result.show()

+---------+-------------------+--------------+------------+
|player_id|      sideline_type|sideline_start|sideline_end|
+---------+-------------------+--------------+------------+
|      276|          Suspended|    2020-02-26|  2020-03-01|
|      276|   Hip/Thigh Injury|    2020-02-02|  2020-02-10|
|      276|Groin/Pelvis Injury|    2019-10-11|  2019-11-20|
|      276|  Ankle/Foot Injury|    2019-08-01|  2019-08-23|
|      276|          Suspended|    2019-05-15|  2019-05-27|
|      276|  Ankle/Foot Injury|    2019-01-24|  2019-04-20|
|      276|       Groin Strain|    2018-12-03|  2019-01-02|
|      276|       Groin Strain|    2018-11-21|  2018-11-27|
|      276|         Broken Toe|    2018-02-26|  2018-05-20|
|      276|Thigh Muscle Strain|    2018-01-20|  2018-01-25|
|      276|         Rib Injury|    2018-01-11|  2018-01-16|
|      276|          Suspended|    2017-12-05|  2017-12-11|
|      276|Thigh Muscle Strain|    2017-11-03|  2017-11-15|
|      276|          Suspended|    2017-

In [141]:
player_sideline.createOrReplaceTempView("player_sideline")

In [150]:
player_sideline = spark.sql("""WITH 
                            df_player_sideline as (SELECT 
                            parameters.player AS player_id,
                            EXPLODE(response) AS expanded_sideline,
                            MONOTONICALLY_INCREASING_ID() AS index_player
                            FROM player_sideline)
                            SELECT
    player_id,
    expanded_sideline.type AS sideline_type,
    expanded_sideline.start AS sideline_start,
    expanded_sideline.end AS sideline_end
FROM
    df_player_sideline;
""")

In [152]:
player_sideline.show()
    


+---------+-------------------+--------------+------------+
|player_id|      sideline_type|sideline_start|sideline_end|
+---------+-------------------+--------------+------------+
|      276|          Suspended|    2020-02-26|  2020-03-01|
|      276|   Hip/Thigh Injury|    2020-02-02|  2020-02-10|
|      276|Groin/Pelvis Injury|    2019-10-11|  2019-11-20|
|      276|  Ankle/Foot Injury|    2019-08-01|  2019-08-23|
|      276|          Suspended|    2019-05-15|  2019-05-27|
|      276|  Ankle/Foot Injury|    2019-01-24|  2019-04-20|
|      276|       Groin Strain|    2018-12-03|  2019-01-02|
|      276|       Groin Strain|    2018-11-21|  2018-11-27|
|      276|         Broken Toe|    2018-02-26|  2018-05-20|
|      276|Thigh Muscle Strain|    2018-01-20|  2018-01-25|
|      276|         Rib Injury|    2018-01-11|  2018-01-16|
|      276|          Suspended|    2017-12-05|  2017-12-11|
|      276|Thigh Muscle Strain|    2017-11-03|  2017-11-15|
|      276|          Suspended|    2017-

In [ ]:
coach_sideline = spark.read.json("file:///home/jss/app/spark/data/soccer_data/sidelined/sidelined_sample.json",multiLine=True)

In [ ]:
coach_sideline.createOrReplaceTempView("coach_sideline")

In [ ]:
coach_sideline = spark.sql("""WITH 
                            df_coach_sideline as (SELECT 
                            parameters.coach AS player_id,
                            EXPLODE(response) AS expanded_sideline,
                            MONOTONICALLY_INCREASING_ID() AS index_coach
                            FROM coach_sideline)
                            SELECT
    coach_id,
    expanded_sideline.type AS sideline_type,
    expanded_sideline.start AS sideline_start,
    expanded_sideline.end AS sideline_end
FROM
    df_coach_sideline;
""")